In [1]:
import sys, os, time
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from gensim.models.wrappers import FastText
import gensim
import fastText
torch.set_printoptions(linewidth=120)
np.set_printoptions(linewidth=120, suppress=True)

/home/fleeb/anaconda3/envs/py36/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
table = torch.load('../fast_table.pth.tar')
table.keys()

dict_keys(['elements', 'vecs', 'rows'])

In [3]:
word2vec = dict(zip(table['elements'], table['vecs']))

In [4]:
X = []
for row in table['rows']:
    X.append(np.concatenate([word2vec[x] for x in row]))
X = torch.from_numpy(np.stack(X)).float()
X.shape

torch.Size([282594, 900])

In [5]:
oie = torch.load('../oie_table.pth.tar')

In [6]:
word2vec = dict(zip(oie['elements'], oie['vecs']))

In [7]:
Y = []
for row in oie['rows']:
    Y.append(np.concatenate([word2vec[x] for x in row]))
Y = torch.from_numpy(np.stack(Y)).float()
Y.shape

torch.Size([3793, 900])

In [8]:
def cosd(x,y):
    if x.ndimension() == 1:
        x = x.unsqueeze(0)
    if y.ndimension() == 1:
        y = y.unsqueeze(0)
    x = F.normalize(x, 2, -1)
    y = F.normalize(y, 2, -1)
    return -x @ y.transpose(-1,-2)/2+.5
def l2(x,y):
    if x.ndimension() == 1:
        x = x.unsqueeze(0)
    if y.ndimension() == 1:
        y = y.unsqueeze(0)
    x = x.unsqueeze(-2)
    y = y.unsqueeze(-3)
    return (x-y).pow(2).mean(-1)

In [ ]:
step = 100
N = len(X)//step
D = []
dist_fn = l2
for i in range(N):
    start = time.time()
    D.append(dist_fn(X[i*step:(i+1)*step], Y))
    if i % 10 ==0:
        print('{}/{} {:.4f}'.format(i+1,N, time.time()-start))
    
#D = torch.cat(D)
#D.shape
len(D)

1/2825 3.5261
11/2825 3.2596
21/2825 3.1955
31/2825 3.2305
41/2825 3.1767
51/2825 3.2734
61/2825 3.1280
71/2825 3.4003
81/2825 3.3374
91/2825 2.7001
101/2825 3.6292
111/2825 3.3079
121/2825 3.6398
131/2825 4.1380
141/2825 4.4164
151/2825 4.0529
161/2825 4.1734
171/2825 4.1854
181/2825 2.9403
191/2825 4.0825
201/2825 3.4101
211/2825 3.9970
221/2825 3.6527
231/2825 3.5535
241/2825 3.4749
251/2825 3.1900
261/2825 3.8819
271/2825 3.8561
281/2825 3.7243
291/2825 3.7539
301/2825 3.6800
311/2825 3.5386
321/2825 3.4891
331/2825 4.0070
341/2825 4.0178
351/2825 3.6068
361/2825 2.4361
371/2825 2.2957
381/2825 2.1924
391/2825 2.2336
401/2825 2.3418
411/2825 2.3611
421/2825 2.3720
431/2825 2.3098
441/2825 2.3404
451/2825 2.3837
461/2825 2.2881
471/2825 2.4684
481/2825 2.4510
491/2825 2.5253
501/2825 2.4827
511/2825 2.6526
521/2825 2.5579
531/2825 2.3617
541/2825 2.3797
551/2825 2.2987
561/2825 2.4152
571/2825 2.6080
581/2825 2.4728
591/2825 2.4722
601/2825 2.4842
611/2825 2.3149
621/2825 2.3765
631